<a href="https://colab.research.google.com/github/agervail/shape/blob/master/Classification_electrique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Classification des éléments électriques d'un plan dxf

Nous utilisons la librarie tensorflow avec l'API haut niveau keras [tf.keras](https://www.tensorflow.org/guide/keras), qui permet de construire et d'entraîner des modèles.

In [0]:
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
 
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

## Récupération des jeux de données


In [0]:
import imageio

base_url = 'https://github.com/agervail/shape/raw/master/img/'
file_path = ['APP.png', 'APPEXT.png', 'INTERDOUBLE.png', 'INTERSIMPLE.png', 'INTERTELERUPTEUR.png', 
               'INTERVV.png', 'LUM.png', 'PRISEEXT.png', 'PRISES16A.png', 'PRISESTEL.png']
train_images_arr = []
for file in file_path:
    im = imageio.imread(base_url + 'train/' + file)
    train_images_arr.append(im[:,:,0])
    
train_images = np.array(train_images_arr)
print(train_images.shape)

In [0]:
file_path = ['APP.png', 'INTERDOUBLE.png', 'INTERSIMPLE.png', 'INTERVV.png',
             'LUM.png', 'PRISEEXT.png', 'PRISES16A.png', 'PRISESTEL.png']
test_images_arr = []
for file in file_path:
    im = imageio.imread(base_url + 'test/' + file)
    if(len(im.shape) == 2):
        test_images_arr.append(im)
    else:
        test_images_arr.append(im[:,:,0]) 
    
test_images = np.array(test_images_arr)
print(test_images.shape)

In [0]:
'''
import imageio

from google.colab import drive
import glob
drive.mount('/content/drive/')

file_path = glob.glob("/content/drive/My Drive/showyou/train/***.png")
file_path.sort()
train_images_arr = []
for file in file_path:
    im = imageio.imread(file)
    train_images_arr.append(im[:,:,0])
    print(file)
    print(im.shape)  
    
train_images = np.array(train_images_arr)
'''

In [0]:
'''
file_path = glob.glob("/content/drive/My Drive/showyou/test/***.png")
file_path.sort()
test_images_arr = []
for file in file_path:
    im = imageio.imread(file)
    print(file)
    print(im.shape)
    if(len(im.shape) == 2):
        test_images_arr.append(im)
    else:
        test_images_arr.append(im[:,:,0])
      
    
test_images = np.array(test_images_arr)
'''

Le jeux de données est téléchargé depuis le drive.

La librairie imageio permet de transformer les png en tableaux numpy.

Ces images étant en noir et blancs nous ne conservons qu'un seul canal `(im[:,:,0])`

* `train_images` contient les images sur lequel le modèle va apprendre
* `test_images` contient les images que l'on veut tester avec le modèle

Les images font 28x28 et les valeurs sont des niveaux de gris entre 0 et 255.

Les *labels* contiennent un indice qui représente l'objet présent dans chaque image.

<table>
  <tr>
    <th>Label</th>
    <th>Classe</th> 
  </tr>
  <tr>
    <td>0</td>
    <td>APP</td> 
  </tr>
  <tr>
    <td>1</td>
    <td>APP_EXT</td> 
  </tr>
    <tr>
    <td>2</td>
    <td>INTERDOUBLE</td> 
  </tr>
    <tr>
    <td>3</td>
    <td>INTERSIMPLE</td> 
  </tr>
    <tr>
    <td>4</td>
    <td>INTERTELERUPTEUR</td> 
  </tr>
    <tr>
    <td>5</td>
    <td>INTERVV</td> 
  </tr>
    <tr>
    <td>6</td>
    <td>LUM</td> 
  </tr>
    <tr>
    <td>7</td>
    <td>PRISEEXT</td> 
  </tr>
    <tr>
    <td>8</td>
    <td>PRISES16A</td> 
  </tr>
    <tr>
    <td>9</td>
    <td>PRISESTEL</td> 
  </tr>
</table>

In [0]:
class_names = ['APP', 'APPEXT', 'INTERDOUBLE', 'INTERSIMPLE', 'INTERTELERUPTEUR', 
               'INTERVV', 'LUM', 'PRISEEXT', 'PRISES16A', 'PRISESTEL']
train_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
test_labels = [0, 2, 3, 5, 6, 7, 8, 9]


## Contenu des données

Taille du jeux d'entraînement et du jeux de test :

In [0]:
print('train_images : ' + str(train_images.shape))
print('train_labels : ' + str(len(train_labels)))
print('test_images : ' + str(test_images.shape))
print('test_labels : ' + str(len(test_labels)))

## Prétraitement des données

Les données doivent être converties pour avoir des valeurs non plus entre 0 et 255 mais entre 0 et 1.

In [0]:
#train_images = train_images / 255.0
#test_images = test_images / 255.0
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

On affiche quelques images du jeu d'entraînement pour vérifier la cohérence des labels.

In [0]:
plt.figure(figsize=(10,10))
for i in range(10):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Setup the layers

The basic building block of a neural network is the *layer*. Layers extract representations from the data fed into them. And, hopefully, these representations are more meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, like `tf.keras.layers.Dense`, have parameters that are learned during training.

In [0]:
#model = keras.Sequential([
#    keras.layers.Flatten(input_shape=(28, 28, 1)),
#    keras.layers.Dense(128, activation=tf.nn.relu),
#    keras.layers.Dense(10, activation=tf.nn.softmax)
#])

The first layer in this network, `tf.keras.layers.Flatten`, transforms the format of the images from a 2d-array (of 28 by 28 pixels), to a 1d-array of 28 * 28 = 784 pixels. Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely-connected, or fully-connected, neural layers. The first `Dense` layer has 128 nodes (or neurons). The second (and last) layer is a 10-node *softmax* layer—this returns an array of 10 probability scores that sum to 1. Each node contains a score that indicates the probability that the current image belongs to one of the 10 classes.

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's *compile* step:

* *Loss function* —This measures how accurate the model is during training. We want to minimize this function to "steer" the model in the right direction.
* *Optimizer* —This is how the model is updated based on the data it sees and its loss function.
* *Metrics* —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

In [0]:
#model.compile(optimizer='adam', 
#              loss='sparse_categorical_crossentropy',
#              metrics=['accuracy'])

## Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model—in this example, the `train_images` and `train_labels` arrays.
2. The model learns to associate images and labels.
3. We ask the model to make predictions about a test set—in this example, the `test_images` array. We verify that the predictions match the labels from the `test_labels` array. 

To start training,  call the `model.fit` method—the model is "fit" to the training data:

In [0]:
train_images = np.array(train_images_arr) #TO REMOVE
print(train_images.shape) #TO REMOVE
train_images = train_images.astype('float32') #TO REMOVE
train_images = train_images.reshape(train_images.shape + (1,))
test_images = np.array(test_images_arr) #TO REMOVE
print(test_images.shape) #TO REMOVE
test_images = test_images.astype('float32') #TO REMOVE
test_images = test_images.reshape(test_images.shape + (1,))


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=2,
    width_shift_range=2,
    height_shift_range=2,
    #brightness_range=[0.8, 1.5],
    zoom_range=[1.0,1.1],
    rescale=1./255.)

datagen.fit(train_images)
k = datagen.standardize(test_images)

Exemple des transformations appliquées aux données

In [0]:
plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(next(datagen.flow(train_images))[0].squeeze())
plt.show()

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28, 1)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
hist = model.fit_generator(datagen.flow(train_images, train_labels, batch_size=30),
                           steps_per_epoch=100, epochs=20, verbose=0)

In [0]:
fig, axs = plt.subplots(nrows=1, ncols=2, sharex=True)

ax = axs[0]
# Plot training & validation accuracy values
ax.plot(hist.history['acc'])
ax.set_title('Accuracy')
ax.legend(['Train', 'Test'], loc='upper left')

ax = axs[1]
ax.plot(hist.history['loss'])
ax.set_title('Loss')
ax.legend(['Train', 'Test'], loc='upper left')

plt.show()

## Tester le résultat de l'apprentissage

Une fois entraîné on teste le modèle sur les données provenant du jeux de test

In [0]:
test_loss, test_acc = model.evaluate(test_images_std, test_labels)

print('Test accuracy:', test_acc)

## Faire des prédictions

Le modèle entraîné peut maintenant classer les images qu'on lui fournit en entrée.

In [0]:
predictions = model.predict(test_images)

In [0]:
num_rows = 4
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images.squeeze())
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()